In [ ]:
MODEL_LINK = "Felladrin/Llama-160M-Chat-v1"
MODEL_NAME = "Llama-160M-Chat-v1"
FINE_TUNED_MODEL_NAME = "Llama-160M-Chat-v1-fraud-detection"
FINE_TUNED_ONNX_MODEL_NAME = "Llama-160M-Chat-v1-fraud-detection-onnx"

In [ ]:
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl

!pip install -U accelerate
!pip install optimum[exporters] onnxruntime[tools] onnxruntime[quantization]

In [ ]:
import wandb
from google.colab import userdata

wb_token = userdata.get('WANDB_API_KEY')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fraud Detection',
    job_type="training",
    anonymous="allow")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lakpriya2 (lakpriya2-university-of-westminster) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
import torch

base_model = MODEL_LINK

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        trust_remote_code=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
)

Device set to use cuda:0


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the tallest building in the world?"}
]

# Apply the chat template to the messages
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generate the output using the pipeline
outputs = pipe(prompt, max_new_tokens=120, do_sample=True)

# Print the generated text
print(outputs[0]["generated_text"])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the tallest building in the world?<|im_end|>
<|im_start|>assistant
The tallest building in the world is the Walt Disney Building, located in Tallahassee, Florida. It houses Disneyland, a theme park, and a futuristic subway system. It is the tallest of the world and was created using innovative and high-tech designs.<|im_end|>


In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")
df['Class'] = df['Class'].map({1: 'Yes', 0: 'No'})
df.rename(columns={'Class': 'Fraud'}, inplace=True)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Text,Fraud
0,"Dear Friend,I hope this may not constitute sor...",Yes
1,Per your strobe request regarding a june meet ...,No
2,"<DIV>Dear Sir,<BR>&nbsp;<BR>It is with tears t...",Yes
3,See belowAlso Allawi appears to be in.,No
4,Fyi,No


In [ ]:
# Shuffle the DataFrame and select only 2000 rows
df = df.sample(frac=1, random_state=85).reset_index(drop=True).head(4000)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

In [ ]:
# # Chat-style prompt generators
# def generate_train_chat_prompt(data_point, tokenizer):
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that detects financial fraud. Respond with 'Yes' or 'No' only."},
#         {"role": "user", "content": f"Is the following text fraudulent?\n\nText: {data_point['Text']}\nFraud:"},
#         {"role": "assistant", "content": data_point["Fraud"]}
#     ]
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
#     return prompt.strip()

# def generate_test_chat_prompt(data_point, tokenizer):
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that detects financial fraud. Respond with 'Yes' or 'No' only."},
#         {"role": "user", "content": f"Is the following text fraudulent?\n\nText: {data_point['Text']}\nFraud:"}
#     ]
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     return prompt.strip()

# # Generate prompts with tokenizer
# X_train['text'] = X_train.apply(lambda row: generate_train_chat_prompt(row, tokenizer), axis=1)
# X_eval['text']  = X_eval.apply(lambda row: generate_train_chat_prompt(row, tokenizer), axis=1)

# # Extract test labels and generate test prompts
# y_true = X_test['Fraud']
# X_test = pd.DataFrame(X_test.apply(lambda row: generate_test_chat_prompt(row, tokenizer), axis=1), columns=["text"])


In [ ]:
def generate_prompt(data_point):
 return f"""
 Is the following text fraudulent? Answer Yes or No only.
Text: {data_point["Text"]}
Fraud: {data_point["Fraud"]}""".strip()

def generate_test_prompt(data_point):
 return f"""
 Is the following text fraudulent? Answer Yes or No only.
Text: {data_point["Text"]}
Fraud: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test['Fraud']
X_test = pd. DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

<ipython-input-11-59588824032c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
<ipython-input-11-59588824032c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

train_data['text'][3]

'Is the following text fraudulent? Answer Yes or No only.\nText: ctikoliingtonVo\'tNEWS I POLITICS I OPINIONS I BUSINESS I LOCAL I SPORTS I ARTS & LIVING I GOING OUT GUIDE I JOBS I CARS I REAL ESTATESHOPPINGAd•••■•■■•.. • us. klReasonable people can disagree on whatever it wasBy Al KamenMonday October 19 2009They call it the World\'s Greatest Deliberative Body (WGDB) Part I.The Senate Judiciary Committee last week was reviewing an amendment by Sen. Richard Durbin (D-I11.) to theUSA Patriot Act to clarify the legal standard needed for an investigation under the act. The amendment wasintended to provide greater protections from abuse by investigators.His colleague Sen. Amy Klobuchar (D-Minn.) spoke in opposition agreeing with committee Chairman PatrickLeahy (D-Vt.) and ranking Republican Jeff Sessions (Ala.) that additional protections are not needed."I would just point to the actual language in here" Klobuchar said "which is it\'s not like this is some pie-in-the-sky standard here. I me

In [ ]:
base_model_name = MODEL_LINK

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    return_dict=True,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/650M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
torch.cuda.empty_cache()

def predict(test, model, tokenizer):
    from transformers import pipeline
    from tqdm import tqdm

    y_pred = []
    categories = ["Yes", "No"]

    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=10,
        temperature=0.1
    )

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        result = pipe(prompt)

        # Remove the prompt from the generated text
        generated_text = result[0]['generated_text']
        answer = generated_text[len(prompt):].strip()


        print("prompt:" + prompt)
        print("answer:" + answer)

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

y_pred_base = predict(X_test, model, tokenizer)
y_pred_base

Device set to use cuda:0
  0%|          | 1/378 [00:01<06:17,  1.00s/it]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: FROM THE DESK OFTHE DIRECTOR=28CEO=29MR=2EHAMDALA RAHAMA=2C OF DUBAI CHAMBER OF COMMERCESIR=2C I GOT YOUR MAIL ADDRESS ON MY RESEARCH IN MY OFFICE IN DUBAI SIR=2FMADAM=2C I WANT TO BRING YOU INTO A BUSINESSTHAT WORTH OF $30M USD DOLLARS=2E IF ONLY YOU WILL BE HONEST AND SINCERE MAKE IT REMAIN CONFIDENTIAL=2E AND IT IS ALL THE ABOVE MENTION AMOUNT=2CTHAT I DISCOVERED AS AN OUTSTANDING SUM=2CTHAT WAS MISSCALCULATED AND INCLUDED AS PART PAYMENT TO OUR FOREGN CONTRACTORS PAYMENT=2E I WAS APPOINTED THE DIRECTOR IN CHARGE OF THIS PAYMENT=2EAS THIS OPPORTUNITY CAME UP=2CI IMMIDIATLY HAVE DECIDED TO MAKE GOOD USE OF IT=2CTO CARRY OUT THIS FUND INTO A FOREIGN ACCOUNT=2CAS WE HERE IN THE MINISTRY ARE NOT ALLOWED TO OPERATE FOREIGN ACCOUNT=2C HENCE ISEEK YOUR ASSISTANCETRANSFER THIS FUND=2C TO FACILITATE THIS BUSINESS=2C INCLUDING YOUR DETAILADDRESS=2C WE HAVE RESOLVED TO OFFER YOU 40%OF THE MONEY FOR YOUR ASSISTANCE AND 5% OF 

  1%|          | 3/378 [00:01<02:23,  2.61it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Dear=2CSubjec=3A Please save me=2EI would like to apply through this medium for your co-operation and to secure an opportunity to invest and do joint business with you in your country=2Cas i do not have much knowleagde about international business=2Finvestment=2EI have a substantial capital I honorably intend to invest in your countryinto a very lucrative venture of which you are to advise and execute thesaid venture over there for the mutual benefits of both of us=2E Your ableco-operation is to become my business partner=2Ctrustee and representativein your country and create an ideas on how money will be invested=2C probably managed and the type ofinvestment after the money is transferred to your custody with yourassistance=2EMeanwhile=2C on indication of your willingness to handle this transactionsincerely by protecting our interests and upon your acceptance of thisproposal=2C I would furnish you with the full deta

  1%|▏         | 5/378 [00:01<01:37,  3.83it/s]

...

 10%|▉         | 37/378 [00:07<01:02,  5.48it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Hello My Good Friend,      Please pardon my indignation for contacting you via this media. I have no option for now than to mail you. I know that my letter may seem strange to you, but I pray God that you give it a fair hearing as you will benefit from it too.      I am in need of your assistance.....My name is Capt.King Noha, I am in the Engineering Military Unit of Liberia.....I have about $23 Million US dollars I made during the civil war in my country on diamond deal and I have moved this fund to Ghana and now I want to move this fund out of that country to your country.      I want to move this fund to your country through diplomatic means, to send it directly to you or anyplace of your choice in your country with diplomatic courier service,.....provided you will assure me of your capability to secure this fund......untill I will come with my investment plans on my own share of the fund.      Once the fund get t

 10%|█         | 38/378 [00:07<01:10,  4.83it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: FROM=3ADr=2E Tony Williams=28Atlantis mining Co=2E=29 =28C=2EE=2EO=29A=2EM=2EC=2E =28LTD=29 34-45 MACOLAR ACCRA GHANA=2E TEL=3A + 233 29325352 FAX=3A + 233 29784574 Direct=3A + 233 24 4758276 E-MAIL=3A tonylinks=40freeghana=2Ecom HELLO SIR=2C I SAW YOU ON A WEB SITE=2C AND I HAVE GONE THROUGH YOUR PROFILE FOUND OUT YOU CAN BE MY PARTNER IN MARKETING MY GOLD AND DIAMOND=2C AM SURE YOU CAN HANDLE IT=2C BUT IF YOU CANT=2C LOOK FOR A BUYER=2C I HAVE PERCENTAGE FOR YOU=2E Please dear=2C I hope I do not offend you for taking your time to read this mail=2E  Am looking for a partner who will be my sells representative abroad =28gold and diamond=29 please let me know if you have any buyer or agent who will be selling the =28gold and diamond=29 whenever I ship=2E When replying=2C please let me know your % percentage in order to help me locate the sellers=2E However=2C I will be more than happy to have you as my partner to repr

KeyboardInterrupt: 

In [ ]:
import psutil
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    auc
)
import time
import matplotlib.pyplot as plt
import seaborn as sns


def evaluate(y_true, y_pred, labels=None, measure_time=False, measure_memory=False):
    if labels is None:
        labels = list(set(y_true))  # Dynamically handle labels if not provided

    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Measure inference time if required
    start_time = time.time()
    if measure_time:
        for _ in range(100):  # Simulate repeated inference for more reliable timing
            _ = np.vectorize(map_func)(y_pred)
    inference_time = time.time() - start_time if measure_time else None

    # Measure memory usage if required
    memory_usage = None
    if measure_memory:
        process = psutil.Process()
        memory_usage = process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB

    # Accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)

    # Accuracy per label
    unique_labels = set(y_true_mapped)
    label_accuracies = {}
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        label_accuracies[labels[label]] = label_accuracy

    # Classification report
    class_report = classification_report(
        y_true=y_true_mapped,
        y_pred=y_pred_mapped,
        target_names=labels,
        labels=list(range(len(labels))),
        output_dict=True  # Return as dictionary for better parsing
    )

    # Confusion matrix
    conf_matrix = confusion_matrix(
        y_true=y_true_mapped,
        y_pred=y_pred_mapped,
        labels=list(range(len(labels)))
    )

    # ROC-AUC and PR-AUC (for binary classification)
    if len(unique_labels) == 2:
        auc_roc = roc_auc_score(y_true_mapped, y_pred_mapped)
        precision, recall, _ = precision_recall_curve(y_true_mapped, y_pred_mapped)
        auc_pr = auc(recall, precision)
    else:
        auc_roc, auc_pr = None, None  # Not applicable for multi-class

    # Create a dictionary of metrics
    metrics = {
        "overall_accuracy": accuracy,
        "label_accuracies": label_accuracies,
        "classification_report": class_report,
        "confusion_matrix": conf_matrix,
        "roc_auc": auc_roc,
        "pr_auc": auc_pr,
        "inference_time": inference_time,
        "memory_usage": memory_usage
    }

    return metrics

labels = ["Yes", "No"]

# Call the function
base_metrics = evaluate(y_true, y_pred_base, labels=labels, measure_time=True, measure_memory=True)

# Access and print metrics
print("Overall Accuracy:", base_metrics["overall_accuracy"])
print("Accuracy per Label:", base_metrics["label_accuracies"])
print("Classification Report:")
print(base_metrics["classification_report"])
print("Confusion Matrix:")
print(base_metrics["confusion_matrix"])
print("ROC-AUC:", base_metrics["roc_auc"])
print("PR-AUC:", base_metrics["pr_auc"])
print("Inference Time:", base_metrics["inference_time"])
print("Memory Usage (MB):",base_metrics['memory_usage'])

Overall Accuracy: 0.005291005291005291
Accuracy per Label: {'Yes': 0.0, 'No': 0.011560693641618497}
Classification Report:
{'Yes': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 205.0}, 'No': {'precision': 0.5, 'recall': 0.011560693641618497, 'f1-score': 0.022598870056497175, 'support': 173.0}, 'micro avg': {'precision': 0.4, 'recall': 0.005291005291005291, 'f1-score': 0.010443864229765013, 'support': 378.0}, 'macro avg': {'precision': 0.25, 'recall': 0.005780346820809248, 'f1-score': 0.011299435028248588, 'support': 378.0}, 'weighted avg': {'precision': 0.22883597883597884, 'recall': 0.005291005291005291, 'f1-score': 0.010342869099931247, 'support': 378.0}}
Confusion Matrix:
[[0 2]
 [1 2]]
ROC-AUC: 0.5037642746369659
PR-AUC: 0.5315151237116555
Inference Time: 0.014084339141845703
Memory Usage (MB): 3295.953125


In [ ]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['v_proj', 'o_proj', 'k_proj', 'q_proj', 'up_proj', 'down_proj', 'gate_proj']

In [ ]:
train_data[3]

{'text': 'Is the following text fraudulent? Answer Yes or No only.\nText: ctikoliingtonVo\'tNEWS I POLITICS I OPINIONS I BUSINESS I LOCAL I SPORTS I ARTS & LIVING I GOING OUT GUIDE I JOBS I CARS I REAL ESTATESHOPPINGAd•••■•■■•.. • us. klReasonable people can disagree on whatever it wasBy Al KamenMonday October 19 2009They call it the World\'s Greatest Deliberative Body (WGDB) Part I.The Senate Judiciary Committee last week was reviewing an amendment by Sen. Richard Durbin (D-I11.) to theUSA Patriot Act to clarify the legal standard needed for an investigation under the act. The amendment wasintended to provide greater protections from abuse by investigators.His colleague Sen. Amy Klobuchar (D-Minn.) spoke in opposition agreeing with committee Chairman PatrickLeahy (D-Vt.) and ranking Republican Jeff Sessions (Ala.) that additional protections are not needed."I would just point to the actual language in here" Klobuchar said "which is it\'s not like this is some pie-in-the-sky standard h

In [ ]:
output_dir=MODEL_NAME

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=4,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_8bit",
    logging_steps=1,
    learning_rate=2e-5,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    # tokenizer=tokenizer,
)

Converting train dataset to ChatML:   0%|          | 0/3017 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/3017 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3017 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3017 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/377 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/377 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/377 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
151,3.151200,3.435426
302,3.027600,3.333131
453,3.721200,3.296588
604,3.675600,3.283588


TrainOutput(global_step=754, training_loss=3.33158310955336, metrics={'train_runtime': 585.8367, 'train_samples_per_second': 5.15, 'train_steps_per_second': 1.287, 'total_flos': 1112992172398080.0, 'train_loss': 3.33158310955336})

In [ ]:
import wandb

wandb.finish()
model.config.use_cache = True

eval/loss,█▃▂▁
eval/mean_token_accuracy,▁▅▇█
eval/num_tokens,▁▃▆█
eval/runtime,▁███
eval/samples_per_second,█▁▁▁
eval/steps_per_second,█▁▁▁
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█
train/grad_norm,▅▃▂▁▅▁▄▂▃▂▂▃▆▁▂▄▃▄▅▅▂▄▃▃▃▂▂▂▅▃▄▅▃▄▇▂▃▅▅█
train/learning_rate,█████▇▇▇▇▇▇▇▇▇▆▅▅▅▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▇▇▇▆▇▆▆▆▆▅▅▆▄▅▄▇▆▇▇▆▆▃▆▅▄▅▅▇▅▅▄▄▅▅▁▅▅▆▅


In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('Llama-160M-Chat-v1/tokenizer_config.json',
 'Llama-160M-Chat-v1/special_tokens_map.json',
 'Llama-160M-Chat-v1/tokenizer.model',
 'Llama-160M-Chat-v1/added_tokens.json',
 'Llama-160M-Chat-v1/tokenizer.json')

In [ ]:
y_pred_fine = predict(X_test, model, tokenizer)

Device set to use cuda:0
  0%|          | 0/378 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  0%|          | 1/378 [00:00<01:16,  4.93it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Jiloty Lauren C <JilotyLC@state.gov>Tuesday December 29 2009 11:06 AMRE: Call list
Fraud:
answer:No
Fra


  1%|          | 3/378 [00:00<01:15,  4.94it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: ATTN=3ARe Urgent Assistance  Kindly allow me the modesty of introducing myself=2E I am GEORGE KABILA=2Cthe son of the former head of state=2F president of Congo-Kinshasha =28then called Zaire=29 Mr=2E Laurent Desire Kabila=2E  I am contacting you in order to ask for your assistance on this confidential Business proposal with full financial benefit for both of us=2E  Before I go into further details please be informed that I am writing without any other person=28s=29 pre-knowledge of my contacting you on this  transaction=2E Therefore I will appreciate same attitude to be maintained all through=2E  I have the sum of USD55Million from a secret sale of Diamond by my father before he was assassinated by one of his body guard =28Rashid=29 on  January 16th 2001=2C which I will like you to receive on my behalf due to security  reasons=2Cas my narration below will explain=2E But before I continue be well informed that your s

  1%|          | 4/378 [00:00<01:17,  4.83it/s]

...

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Kerry is sleeping
Fraud:
answer:No
Fra
prompt:Is the following text fraudulent? Answer Yes or No only.
Text: If this proposal meet your intrerest, we have decided to give you anegotiable percentage from the entire sum, hoping that it will besatisfactory. Please send along your response the following:Confidential/Private Tel and Fax number.Contact name and address.Thank you for your much anticipated urgent response.Respectfully,Dr.Kelvin Promchay
Fraud:
answer:No, No,


100%|█████████▉| 377/378 [01:16<00:00,  4.94it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Dear Friend=2CFirst=2C I must apologize if my mode of first contact is not acceptable  to you and also solicit yourconfidence in this transaction=3B this is by virtue of its nature as  being utterly private and top secret=2EThough I know that a transaction of this magnitude will make anyonea pprehensive and worried=2C but I am assuring you that you do not have anything to worry as I have perfected every arrangement with my  colleague =2E I decided to contact you due to the urgency of this transaction=2C as I am aware of the high sense of responsibility placed  on  you and believe  in your discreetness and ability to handle transaction of this nature=2ELet me start by introducing my self properly to you=2E I am SILVA FALL  =28Chief Executive Officer=29 of my Financial Consortium here in Daka- Senegal =2EI  came to know and  pick on you in my Private Search for a Reliable and ReputablePerson to  handle this Confidentia

100%|██████████| 378/378 [01:16<00:00,  4.91it/s]

prompt:Is the following text fraudulent? Answer Yes or No only.
Text: Dear sir I am Mark miller=2C the only son of late former Director of finance Chief late Dr Jonathan miller=2C of Sierra-Leone diamond and mining corporation=2C i hope you are doing well=2E I am contacting you with the hope that you will be of great assistance to me I currently have within my reach the sum of =28$10 Million=29 Ten Million Dollars=2Ewas deposited by my late father in security company in Dakar Senegal=2C my name appears as next of kin=2E this money was realised from diamonds export=2C now i seeking asylum in Dakar-Senegal under =28UNHCR=29=2E I Contacted You Because Of The Need=2C which I intend to use for investment purposes=2E therefore personally appeal to you seriously and religiously for your urgent assistance to move this money into your country=2C i leave my country due to the disastrous civil war in my country=2C Please kindly reply me by this mail and include your direct telephone numbers where

In [ ]:
fine_tuned_metrics = evaluate(y_true, y_pred_fine, labels=labels, measure_time=True, measure_memory=True)

# Access and print metrics
print("Overall Accuracy:", fine_tuned_metrics["overall_accuracy"])
print("Accuracy per Label:", fine_tuned_metrics["label_accuracies"])
print("Classification Report:")
print(fine_tuned_metrics["classification_report"])
print("Confusion Matrix:")
print(fine_tuned_metrics["confusion_matrix"])
print("ROC-AUC:", fine_tuned_metrics["roc_auc"])
print("PR-AUC:", fine_tuned_metrics["pr_auc"])
print("Inference Time:", fine_tuned_metrics["inference_time"])
print("Memory Usage (MB):",fine_tuned_metrics['memory_usage'])

Overall Accuracy: 0.8544973544973545
Accuracy per Label: {'Yes': 0.8585365853658536, 'No': 0.8497109826589595}
Classification Report:
{'Yes': {'precision': 0.8844221105527639, 'recall': 0.8585365853658536, 'f1-score': 0.8712871287128713, 'support': 205.0}, 'No': {'precision': 0.8497109826589595, 'recall': 0.8497109826589595, 'f1-score': 0.8497109826589595, 'support': 173.0}, 'micro avg': {'precision': 0.8682795698924731, 'recall': 0.8544973544973545, 'f1-score': 0.8613333333333333, 'support': 378.0}, 'macro avg': {'precision': 0.8670665466058617, 'recall': 0.8541237840124065, 'f1-score': 0.8604990556859153, 'support': 378.0}, 'weighted avg': {'precision': 0.8685358006966047, 'recall': 0.8544973544973545, 'f1-score': 0.8614123317093615, 'support': 378.0}}
Confusion Matrix:
[[176  26]
 [ 23 147]]
ROC-AUC: 0.8549696884252079
PR-AUC: 0.8806520328183425
Inference Time: 0.01407933235168457
Memory Usage (MB): 3635.77734375


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_metrics(metrics, labels):
    # Plot overall accuracy
    plt.figure(figsize=(6, 4))
    plt.bar(["Overall Accuracy"], [metrics["overall_accuracy"]], color="skyblue")
    plt.ylim(0, 1)
    plt.ylabel("Accuracy")
    plt.title("Overall Accuracy")
    for i, v in enumerate([metrics["overall_accuracy"]]):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

    # Plot accuracy per label
    label_names = list(metrics["label_accuracies"].keys())
    label_accuracies = list(metrics["label_accuracies"].values())

    plt.figure(figsize=(8, 6))
    bars = plt.bar(label_names, label_accuracies, color="lightgreen")
    plt.ylim(0, 1)
    plt.ylabel("Accuracy")
    plt.title("Accuracy per Label")
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2.0, height + 0.02, f"{height:.2f}", ha="center", fontsize=10)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

    # Plot confusion matrix
    conf_matrix = metrics["confusion_matrix"]

    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

    # Classification report visualization (precision, recall, f1-score)
    class_report = metrics["classification_report"]
    report_labels = labels
    precision = [class_report[label]["precision"] for label in report_labels]
    recall = [class_report[label]["recall"] for label in report_labels]
    f1_score = [class_report[label]["f1-score"] for label in report_labels]

    x = np.arange(len(report_labels))  # Label locations
    width = 0.25  # Bar width

    plt.figure(figsize=(10, 6))
    bars1 = plt.bar(x - width, precision, width, label="Precision", color="coral")
    bars2 = plt.bar(x, recall, width, label="Recall", color="skyblue")
    bars3 = plt.bar(x + width, f1_score, width, label="F1-Score", color="lightgreen")

    plt.xticks(x, report_labels)
    plt.ylim(0, 1)
    plt.xlabel("Labels")
    plt.ylabel("Score")
    plt.title("Classification Metrics by Label")
    plt.legend()

    # Add annotations
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width() / 2.0, height + 0.02, f"{height:.2f}", ha="center", fontsize=10)

    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

    # If ROC-AUC and PR-AUC exist, plot them
    if "roc_auc" in metrics and metrics["roc_auc"] is not None:
        plt.figure(figsize=(6, 4))
        plt.bar(["ROC-AUC"], [metrics["roc_auc"]], color="mediumpurple")
        plt.ylim(0, 1)
        plt.ylabel("Score")
        plt.title("ROC-AUC Score")
        for i, v in enumerate([metrics["roc_auc"]]):
            plt.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
        plt.grid(axis="y", linestyle="--", alpha=0.7)
        plt.show()

    if "pr_auc" in metrics and metrics["pr_auc"] is not None:
        plt.figure(figsize=(6, 4))
        plt.bar(["PR-AUC"], [metrics["pr_auc"]], color="gold")
        plt.ylim(0, 1)
        plt.ylabel("Score")
        plt.title("PR-AUC Score")
        for i, v in enumerate([metrics["pr_auc"]]):
            plt.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
        plt.grid(axis="y", linestyle="--", alpha=0.7)
        plt.show()


In [ ]:
# Plot the metrics
plot_metrics(fine_tuned_metrics, labels)

In [ ]:
def plot_overall_accuracy_comparison(base_metrics, fine_tuned_metrics):
    models = ['Base Model', 'Fine-Tuned Model']
    accuracies = [base_metrics["overall_accuracy"], fine_tuned_metrics["overall_accuracy"]]

    plt.figure(figsize=(8, 4))
    plt.bar(models, accuracies, color=['skyblue', 'lightgreen'])
    plt.ylim(0, 1)
    plt.ylabel("Accuracy")
    plt.title("Overall Accuracy Comparison")
    plt.show()


In [ ]:
def plot_label_accuracy_comparison(base_metrics, fine_tuned_metrics, labels, base_model_name, fine_tuned_model_name):
    base_accuracies = [base_metrics["label_accuracies"][label] for label in labels]
    fine_tuned_accuracies = [fine_tuned_metrics["label_accuracies"][label] for label in labels]

    x = np.arange(len(labels))  # Label positions
    width = 0.35  # Bar width

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, base_accuracies, width, label=base_model_name, color='skyblue')
    plt.bar(x + width/2, fine_tuned_accuracies, width, label=fine_tuned_model_name, color='lightgreen')

    plt.xticks(x, labels)
    plt.ylim(0, 1)
    plt.ylabel("Accuracy")
    plt.title("Label-Wise Accuracy Comparison")
    plt.legend()
    plt.show()


In [ ]:
def plot_confusion_matrix_comparison(base_metrics, fine_tuned_metrics, labels, base_model_name, fine_tuned_model_name):
    base_conf_matrix = base_metrics["confusion_matrix"]
    fine_tuned_conf_matrix = fine_tuned_metrics["confusion_matrix"]

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    sns.heatmap(base_conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, ax=axes[0])
    axes[0].set_ylabel("True Labels")
    axes[0].set_title( base_model_name + " Confusion Matrix")
    axes[0].set_xlabel("Predicted Labels")
    axes[0].set_ylabel("True Labels")

    sns.heatmap(fine_tuned_conf_matrix, annot=True, fmt="d", cmap="Greens", xticklabels=labels, yticklabels=labels, ax=axes[1])
    axes[1].set_title(fine_tuned_model_name + " Confusion Matrix")
    axes[1].set_xlabel("Predicted Labels")
    axes[1].set_ylabel("True Labels")

    plt.tight_layout()
    plt.show()


In [ ]:
def plot_classification_metrics_comparison(base_metrics, fine_tuned_metrics, labels, base_model_name, fine_tuned_model_name ):
    metrics = ["precision", "recall", "f1-score"]
    x = np.arange(len(labels))  # Label positions
    width = 0.3  # Bar width

    for metric in metrics:
        base_scores = [base_metrics["classification_report"][label][metric] for label in labels]
        fine_tuned_scores = [fine_tuned_metrics["classification_report"][label][metric] for label in labels]

        plt.figure(figsize=(10, 6))
        plt.bar(x - width/2, base_scores, width, label=base_model_name, color='skyblue')
        plt.bar(x + width/2, fine_tuned_scores, width, label=fine_tuned_model_name, color='lightgreen')

        plt.xticks(x, labels)
        plt.ylim(0, 1)
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} Comparison by Label")
        plt.legend()
        plt.show()


In [ ]:
def plot_roc_pr_curves(y_true, y_pred_base, y_pred_fine, labels, base_model_name, fine_tuned_model_name):
    from sklearn.metrics import roc_curve, precision_recall_curve, auc

    # Map labels to numeric values for y_true and predictions
    label_mapping = {label: idx for idx, label in enumerate(labels)}  # e.g., {"Yes": 1, "No": 0}

    # Convert y_true to numeric
    y_true_numeric = np.array([label_mapping[label] for label in y_true])

    # Convert predictions to numeric
    y_pred_base_numeric = np.array([label_mapping[label] for label in y_pred_base])
    y_pred_fine_numeric = np.array([label_mapping[label] for label in y_pred_fine])

    # Set the positive label explicitly for "Yes" (1)
    pos_label = label_mapping["Yes"]

    # ROC Curve
    plt.figure(figsize=(8, 6))
    for y_pred, name in zip([y_pred_base_numeric, y_pred_fine_numeric], [base_model_name, fine_tuned_model_name]):
        fpr, tpr, _ = roc_curve(y_true_numeric, y_pred, pos_label=pos_label)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC: {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], linestyle="--", color="grey")
    plt.title("ROC Curve Comparison")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.grid()
    plt.show()

    # PR Curve
    plt.figure(figsize=(8, 6))
    for y_pred, name in zip([y_pred_base_numeric, y_pred_fine_numeric], [base_model_name, fine_tuned_model_name]):
        precision, recall, _ = precision_recall_curve(y_true_numeric, y_pred, pos_label=pos_label)
        pr_auc = auc(recall, precision)
        plt.plot(recall, precision, label=f"{name} (AUC: {pr_auc:.2f})")
    plt.title("Precision-Recall Curve Comparison")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plot_efficiency_comparison(base_metrics, fine_tuned_metrics, base_model_name, fine_tuned_model_name ):
    models = [base_model_name, fine_tuned_model_name]
    times = [base_metrics.get("inference_time", 0), fine_tuned_metrics.get("inference_time", 0)]
    memory = [base_metrics.get("memory_usage", 0), fine_tuned_metrics.get("memory_usage", 0)]

    # Latency Comparison
    plt.figure(figsize=(8, 4))
    plt.bar(models, times, color="skyblue")
    plt.ylabel("Inference Time (s)")
    plt.title("Inference Time Comparison")
    plt.show()

    # Memory Usage Comparison
    plt.figure(figsize=(8, 4))
    plt.bar(models, memory, color="lightgreen")
    plt.ylabel("Memory Usage (MB)")
    plt.title("Memory Usage Comparison")
    plt.show()


In [ ]:
# Plot comparisons
plot_overall_accuracy_comparison(base_metrics, fine_tuned_metrics)
plot_label_accuracy_comparison(base_metrics, fine_tuned_metrics, labels, "Base Model", "Fine-tuned Model")
plot_confusion_matrix_comparison(base_metrics, fine_tuned_metrics, labels, "Base Model", "Fine-tuned Model")
plot_classification_metrics_comparison(base_metrics, fine_tuned_metrics, labels, "Base Model", "Fine-tuned Model")
plot_roc_pr_curves(y_true, y_pred_base, y_pred_fine, labels, "Base Model", "Fine-tuned Model")
plot_efficiency_comparison(base_metrics, fine_tuned_metrics, "Base Model", "Fine-tuned Model")

In [ ]:
from peft import AutoPeftModelForCausalLM

merged_model = AutoPeftModelForCausalLM.from_pretrained(output_dir)
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained(FINE_TUNED_MODEL_NAME)
tokenizer.save_pretrained(FINE_TUNED_MODEL_NAME)

('Llama-160M-Chat-v1-fraud-detection/tokenizer_config.json',
 'Llama-160M-Chat-v1-fraud-detection/special_tokens_map.json',
 'Llama-160M-Chat-v1-fraud-detection/tokenizer.model',
 'Llama-160M-Chat-v1-fraud-detection/added_tokens.json',
 'Llama-160M-Chat-v1-fraud-detection/tokenizer.json')

In [ ]:
from huggingface_hub import hf_hub_download, login, HfApi

login(token=userdata.get('HF_TOKEN'))

In [ ]:
from huggingface_hub import HfApi, HfFolder

# Define your Hugging Face repo ID and the folder path to push
repo_id = "lakpriya/" + FINE_TUNED_MODEL_NAME
folder_path = "/content/" + FINE_TUNED_MODEL_NAME

# Create the repo if it doesn't exist
api = HfApi()
try:
    api.create_repo(repo_id=repo_id, private=False, exist_ok=False)
except:
    print(f"Repository {repo_id} already exists or there was an error creating it.")

# Upload the model
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
    commit_message="Add new files to repository"
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/650M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lakpriya/Llama-160M-Chat-v1-fraud-detection/commit/00ac4b6981bb2ceb70fb70ef0e54d31cb77db1f3', commit_message='Add new files to repository', commit_description='', oid='00ac4b6981bb2ceb70fb70ef0e54d31cb77db1f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lakpriya/Llama-160M-Chat-v1-fraud-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='lakpriya/Llama-160M-Chat-v1-fraud-detection'), pr_revision=None, pr_num=None)

In [ ]:
input_path = '/content/'+FINE_TUNED_MODEL_NAME
output_path = '/content/'+FINE_TUNED_ONNX_MODEL_NAME
repo_path = "lakpriya/" + FINE_TUNED_MODEL_NAME

!optimum-cli export onnx --model {repo_path} --task text-generation {FINE_TUNED_ONNX_MODEL_NAME}

2025-03-28 03:46:26.149081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743133586.171946    6578 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743133586.179036    6578 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
config.json: 100% 746/746 [00:00<00:00, 5.62MB/s]
model.safetensors: 100% 650M/650M [00:07<00:00, 87.4MB/s]
generation_config.json: 100% 132/132 [00:00<00:00, 1.15MB/s]
tokenizer_config.json: 100% 1.20k/1.20k [00:00<00:00, 9.93MB/s]
tokenizer.model: 100% 500k/500k [00:00<00:00, 23.8MB/s]
tokenizer.json: 100% 3.62M/3.62M [00:00<00:00, 12.3MB/s]
special_tokens_map.json: 100% 437/437 [00:00<00:00, 3.86MB/s]
/usr/local/lib/python3.11/dis

In [ ]:
X_test

,text
3394,Is the following text fraudulent? Answer Yes o...
3395,Is the following text fraudulent? Answer Yes o...
3396,Is the following text fraudulent? Answer Yes o...
3397,Is the following text fraudulent? Answer Yes o...
3398,Is the following text fraudulent? Answer Yes o...
...,...
3767,Is the following text fraudulent? Answer Yes o...
3768,Is the following text fraudulent? Answer Yes o...
3769,Is the following text fraudulent? Answer Yes o...
3770,Is the following text fraudulent? Answer Yes o...


In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForCausalLM
from tqdm import tqdm

def predict_onnx(test, model_path, categories=["Yes", "No"]):
    # Load the tokenizer and ONNX model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = ORTModelForCausalLM.from_pretrained(model_path, use_cache=False, use_io_binding=False)

    # Ensure the model is running on the specified provider
    assert model.providers == ['CPUExecutionProvider'], "Model is not running on CPUExecutionProvider"

    y_pred = []

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        inputs = tokenizer(prompt, return_tensors="pt")

        # Generate predictions
        gen_tokens = model.generate(**inputs, max_length=inputs["input_ids"].shape[1] + 10, temperature=0.1)
        result_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

        # Strip the prompt from the beginning
        answer = result_text[len(prompt):].strip()

        print(f"Input: {prompt}")
        print(f"Output: {answer}")

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred


In [ ]:
# Example usage:
model_path = "/content/" + FINE_TUNED_ONNX_MODEL_NAME  # Replace with your ONNX model path
y_pred_onnx = predict_onnx(X_test, model_path)


  0%|          | 0/378 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  0%|          | 1/378 [00:05<32:27,  5.16s/it]

Input: Is the following text fraudulent? Answer Yes or No only.
Text: FROM THE DESK OFTHE DIRECTOR=28CEO=29MR=2EHAMDALA RAHAMA=2C OF DUBAI CHAMBER OF COMMERCESIR=2C I GOT YOUR MAIL ADDRESS ON MY RESEARCH IN MY OFFICE IN DUBAI SIR=2FMADAM=2C I WANT TO BRING YOU INTO A BUSINESSTHAT WORTH OF $30M USD DOLLARS=2E IF ONLY YOU WILL BE HONEST AND SINCERE MAKE IT REMAIN CONFIDENTIAL=2E AND IT IS ALL THE ABOVE MENTION AMOUNT=2CTHAT I DISCOVERED AS AN OUTSTANDING SUM=2CTHAT WAS MISSCALCULATED AND INCLUDED AS PART PAYMENT TO OUR FOREGN CONTRACTORS PAYMENT=2E I WAS APPOINTED THE DIRECTOR IN CHARGE OF THIS PAYMENT=2EAS THIS OPPORTUNITY CAME UP=2CI IMMIDIATLY HAVE DECIDED TO MAKE GOOD USE OF IT=2CTO CARRY OUT THIS FUND INTO A FOREIGN ACCOUNT=2CAS WE HERE IN THE MINISTRY ARE NOT ALLOWED TO OPERATE FOREIGN ACCOUNT=2C HENCE ISEEK YOUR ASSISTANCETRANSFER THIS FUND=2C TO FACILITATE THIS BUSINESS=2C INCLUDING YOUR DETAILADDRESS=2C WE HAVE RESOLVED TO OFFER YOU 40%OF THE MONEY FOR YOUR ASSISTANCE AND 5% OF 

  1%|          | 2/378 [00:08<23:59,  3.83s/it]

Input: Is the following text fraudulent? Answer Yes or No only.
Text: Dear=2CSubjec=3A Please save me=2EI would like to apply through this medium for your co-operation and to secure an opportunity to invest and do joint business with you in your country=2Cas i do not have much knowleagde about international business=2Finvestment=2EI have a substantial capital I honorably intend to invest in your countryinto a very lucrative venture of which you are to advise and execute thesaid venture over there for the mutual benefits of both of us=2E Your ableco-operation is to become my business partner=2Ctrustee and representativein your country and create an ideas on how money will be invested=2C probably managed and the type ofinvestment after the money is transferred to your custody with yourassistance=2EMeanwhile=2C on indication of your willingness to handle this transactionsincerely by protecting our interests and upon your acceptance of thisproposal=2C I would furnish you with the full deta

  1%|          | 3/378 [00:15<33:14,  5.32s/it]

...

Input: Is the following text fraudulent? Answer Yes or No only.
Text: I can't print email so pis fax. Thx.
Fraud:
Output: No. No. No. No. No.


  3%|▎         | 10/378 [00:46<29:06,  4.75s/it]

Input: Is the following text fraudulent? Answer Yes or No only.
Text: Dear Friend=2EMy name is Mr=2E Fredrick Andrew=2E I trained and work as an external auditor for the Development Bank of Singapore =28DBS=29 of Singapore=2E I have taken pains to find your contact through personal endeavours because a late investor =2C who bears the same last name with you has left monies totaling a little over $10 million United States Dollars with Our Bank for the past twelve years and no next of kin has come forward all these years=2E Banking regulation legislation in Singapore demand that I notify the fiscal authorities after a statutory time period of twelve years when dormant accounts of this type are called in by the monetary regulatory bodies=2E The above set of facts underscores my reason of writing=2EI am of the settled conviction that using my insider leverage=2C working with you=2Csince you have the same last Name with the Late Investor=2C we can secure the funds in the account for us inst

  3%|▎         | 10/378 [00:51<31:25,  5.12s/it]


KeyboardInterrupt: 

In [ ]:
onnx_metrics = evaluate(y_true, y_pred_onnx, labels=labels, measure_time=True, measure_memory=True)

# Access and print metrics
print("Overall Accuracy:", fine_tuned_metrics["overall_accuracy"])
print("Accuracy per Label:", fine_tuned_metrics["label_accuracies"])
print("Classification Report:")
print(fine_tuned_metrics["classification_report"])
print("Confusion Matrix:")
print(fine_tuned_metrics["confusion_matrix"])
print("ROC-AUC:", fine_tuned_metrics["roc_auc"])
print("PR-AUC:", fine_tuned_metrics["pr_auc"])
print("Inference Time:", fine_tuned_metrics["inference_time"])
print("Memory Usage (MB):",fine_tuned_metrics['memory_usage'])

In [ ]:
# Define your Hugging Face repo ID and the folder path to push
repo_id = "lakpriya/" + FINE_TUNED_ONNX_MODEL_NAME
folder_path = "/content/" + FINE_TUNED_ONNX_MODEL_NAME

# Create the repo if it doesn't exist
api = HfApi()
try:
    api.create_repo(repo_id=repo_id, private=False, exist_ok=False)
except:
    print(f"Repository {repo_id} already exists or there was an error creating it.")

# Upload the model
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
    commit_message="Add new files to repository"
)

In [ ]:
# Plot comparisons
plot_overall_accuracy_comparison(base_metrics, onnx_metrics)
plot_label_accuracy_comparison(base_metrics, onnx_metrics, labels, "Base Model", "Fine-tuned ONNX Model")
plot_confusion_matrix_comparison(base_metrics, onnx_metrics, labels, "Base Model", "Fine-tuned ONNX Model")
plot_classification_metrics_comparison(base_metrics, onnx_metrics, labels, "Base Model", "Fine-tuned ONNX Model")
plot_roc_pr_curves(y_true, y_pred_base, y_pred_onnx, labels, "Base Model", "Fine-tuned ONNX Model")
plot_efficiency_comparison(base_metrics, onnx_metrics, "Base Model", "Fine-tuned ONNX Model")

In [ ]:
import shutil
import os

# Copy files and folders
source_dir = "/content/" + FINE_TUNED_ONNX_MODEL_NAME
destination_dir = "/content/"+ FINE_TUNED_ONNX_MODEL_NAME +"___quantized"

try:
    shutil.copytree(source_dir, destination_dir)
    print(f"Successfully copied {source_dir} to {destination_dir}")
except FileExistsError:
    print(f"Destination directory {destination_dir} already exists.")
except OSError as e:
    print(f"Error copying directory: {e}")

# Delete specified files in the destination directory
files_to_delete = ["model.onnx", "model.onnx_data"]
for file in files_to_delete:
    file_path = os.path.join(destination_dir, file)
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            print(f"Successfully deleted {file_path}")
        except OSError as e:
            print(f"Error deleting {file_path}: {e}")
    else:
        print(f"File {file_path} not found.")


In [ ]:
import onnx
from onnxruntime.tools.symbolic_shape_infer import SymbolicShapeInference

def preprocess_onnx_model(input_model_path, output_model_path):
    model = onnx.load(input_model_path)

    shape_infer = SymbolicShapeInference.infer_shapes(model, auto_merge=True, verbose=1)

    if shape_infer is None:
        print("Shape inference failed.")
    else:
        onnx.save(shape_infer, output_model_path)
        print(f"Model saved at {output_model_path}")

input_model_path = output_path + '/model.onnx'
output_model_path = output_path + '/model-infer.onnx'

preprocess_onnx_model(input_model_path, output_model_path)

In [ ]:
from onnxruntime.quantization.shape_inference import quant_pre_process

optimized_model_path = output_path + "/model_optimized.onnx"
quant_pre_process(output_path + "/model.onnx", optimized_model_path)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantized_model_path = "/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model_quantized.onnx"
quantize_dynamic("/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model.onnx", quantized_model_path, weight_type=QuantType.QInt8, per_channel=True,
    reduce_range=False, use_external_data_format=True)

In [ ]:
import os

size_in_bytes = os.path.getsize("/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model.onnx") #+ os.path.getsize("/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model.onnx_data")
size_in_mb = size_in_bytes / (1024 ** 2)
print(f"Model size: {size_in_mb:.2f} MB")

size_in_bytes = os.path.getsize("/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model_quantized.onnx") + os.path.getsize("/content/"+FINE_TUNED_ONNX_MODEL_NAME+"/model_quantized.onnx.data")
size_in_mb = size_in_bytes / (1024 ** 2)
print(f"Model size: {size_in_mb:.2f} MB")

In [ ]:
# Example usage:
model_path = "/content/" + FINE_TUNED_ONNX_MODEL_NAME +"___quantized"  # Replace with your ONNX model path
y_pred_onnx_quantized = predict_onnx(X_test, model_path)


In [ ]:
onnx_quantized_metrics = evaluate(y_true, y_pred_onnx_quantized, labels=labels, measure_time=True, measure_memory=True)

# Access and print metrics
print("Overall Accuracy:", onnx_quantized_metrics["overall_accuracy"])
print("Accuracy per Label:", onnx_quantized_metrics["label_accuracies"])
print("Classification Report:")
print(onnx_quantized_metrics["classification_report"])
print("Confusion Matrix:")
print(onnx_quantized_metrics["confusion_matrix"])
print("ROC-AUC:", onnx_quantized_metrics["roc_auc"])
print("PR-AUC:", onnx_quantized_metrics["pr_auc"])
print("Inference Time:", onnx_quantized_metrics["inference_time"])
print("Memory Usage (MB):",onnx_quantized_metrics['memory_usage'])

In [ ]:
# Define your Hugging Face repo ID and the folder path to push
repo_id = "lakpriya/"+ FINE_TUNED_ONNX_MODEL_NAME +"_quantized"
folder_path = "/content/"+ FINE_TUNED_ONNX_MODEL_NAME +"___quantized"

# Create the repo if it doesn't exist
api = HfApi()
try:
    api.create_repo(repo_id=repo_id, private=False, exist_ok=False)
except:
    print(f"Repository {repo_id} already exists or there was an error creating it.")

# Upload the model
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
    commit_message="Add new files to repository"
)

In [ ]:
# Plot comparisons
plot_overall_accuracy_comparison(base_metrics, onnx_quantized_metrics)
plot_label_accuracy_comparison(base_metrics, onnx_quantized_metrics, labels, "Base Model", "Fine-tuned ONNX quantized Model")
plot_confusion_matrix_comparison(base_metrics, onnx_quantized_metrics, labels, "Base Model", "Fine-tuned ONNX quantized Model")
plot_classification_metrics_comparison(base_metrics, onnx_quantized_metrics, labels, "Base Model", "Fine-tuned quantized ONNX Model")
plot_roc_pr_curves(y_true, y_pred_base, y_pred_onnx_quantized, labels, "Base Model", "Fine-tuned ONNX quantized Model")
plot_efficiency_comparison(base_metrics, onnx_quantized_metrics, "Base Model", "Fine-tuned ONNX quantized Model")